In [1]:
print("Hello, World")

Hello, World


In [1]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 56.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.4 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117031 sha256=9a76e287f751caaa8f7322b24159f7d0dc66f15fa9569228855160d8c697f90d
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.6/865.6 kB 23.5 MB/s eta 0:00:0000:01


In [6]:
import os
import cv2
from ultralytics import YOLO
from deepface import DeepFace
from mtcnn import MTCNN

# Paths
video_path = '/kaggle/input/live-videos/videos/video3.mp4'  # Update with your video path in Colab
output_directory = '/kaggle/working/'  # Specify the directory where you want to save the video
output_filename = 'processed_video.mp4'  # Specify the desired output filename
video_path_out = os.path.join(output_directory, output_filename)

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [8]:
# Video capture and writer setup
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_path_out, fourcc, fps, (frame_width, frame_height))

# Load the YOLOv8 model for gender detection
model_path = '/kaggle/input/weight-for-yolo/last.pt'  # Update with your model path in Colab
gender_model = YOLO(model_path)

# Load the MTCNN face detector
face_detector = MTCNN()

# Threshold for gender detection score
threshold = 0.5

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Perform gender detection on the whole frame using YOLOv8
    results = gender_model(frame)[0]

    # Process each detected object (assuming they are all faces)
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            gender_label = f"{results.names[int(class_id)].upper()} {score:.2f}"
            face_region = frame[int(y1):int(y2), int(x1):int(x2)]

            if face_region.size == 0:
                continue

            try:
                # Detect face using MTCNN within the detected region
                faces = face_detector.detect_faces(cv2.cvtColor(face_region, cv2.COLOR_BGR2RGB))

                for face in faces:
                    fx, fy, fw, fh = face['box']
                    face_subregion = face_region[fy:fy+fh, fx:fx+fw]

                    if face_subregion.size == 0:
                        continue

                    # Perform age and emotion detection
                    predictions = DeepFace.analyze(face_subregion, actions=['age', 'emotion'], enforce_detection=False)
                    if isinstance(predictions, list):
                        predictions = predictions[0]

                    age = predictions['age']
                    dominant_emotion = predictions['dominant_emotion']

                    # Draw a single bounding box with gender, age, and emotion
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    cv2.putText(frame, f'Gender: {gender_label}, Age: {age}, Emotion: {dominant_emotion}',
                                (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            except ValueError as e:
                print(f"Error processing face region at [{int(x1), int(y1), int(x2-x1), int(y2-y1)}]: {e}")

    # Write the frame to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()

print(f"Processed video saved to {video_path_out}")


0: 384x640 5 males, 6 females, 84.4ms
Speed: 11.6ms preprocess, 84.4ms inference, 256.8ms postprocess per image at shape (1, 3, 384, 640)
1/1 [==============================] - 1s 901ms/step


Action: age:   0%|          | 0/2 [00:00<?, ?it/s]    

24-08-13 07:09:16 - age_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: /root/.deepface/weights/age_model_weights.h5

  0%|          | 0.00/539M [00:00<?, ?B/s]
  2%|▏         | 11.0M/539M [00:00<00:05, 89.4MB/s]
  6%|▌         | 32.0M/539M [00:00<00:04, 117MB/s] 
 10%|▉         | 53.0M/539M [00:00<00:03, 126MB/s]
 14%|█▎        | 73.9M/539M [00:00<00:03, 126MB/s]
 16%|█▋        | 88.6M/539M [00:00<00:03, 118MB/s]
 19%|█▊        | 101M/539M [00:00<00:04, 103MB/s] 
 22%|██▏       | 116M/539M [00:01<00:04, 99.5MB/s]
 25%|██▌       | 137M/539M [00:01<00:03, 109MB/s] 
 29%|██▉       | 158M/539M [00:01<00:03, 117MB/s]
 32%|███▏      | 170M/539M [00:01<00:03, 110MB/s]
 34%|███▎      | 181M/539M [00:01<00:03, 92.0MB/s]
 37%|███▋      | 199M/539M [00:01<00:03, 110MB/s] 
 39%|███▉      | 211M/539M [00:01<00:03, 101MB/s]
 43%|████▎     | 231M/539M [00:02<00:02, 113MB/s]
 47%|████▋     | 252M/539M [00:02<00:02, 123MB/s]
 50%|████▉     | 268M/539M [00:02<00

24-08-13 07:09:25 - facial_expression_model_weights.h5 will be downloaded...


...

1/1 [==============================] - 0s 18ms/step


Action: emotion: 100%|██████████| 2/2 [00:00<00:00, 11.25it/s]


1/1 [==============================] - 0s 19ms/step


Action: emotion: 100%|██████████| 2/2 [00:00<00:00, 11.53it/s]


Processed video saved to /kaggle/working/processed_video.mp4


In [5]:
import os
import cv2
from ultralytics import YOLO
from deepface import DeepFace
from mtcnn import MTCNN

# Paths
image_directory = "D:\IntegrationXpert\male & female classification (2)\testing\ab"  # Update with your images directory path
output_directory = r"D:\IntegrationXpert\final code\testing"  # Specify the directory where you want to save the processed images

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Load the YOLOv8 model for gender detection
model_path = r"D:\IntegrationXpert\final code\last.pt"
gender_model = YOLO(model_path)

# Load the MTCNN face detector
face_detector = MTCNN()

# Threshold for gender detection score
threshold = 0.5

# Loop through each image in the directory
for image_filename in os.listdir(image_directory):
    image_path = os.path.join(image_directory, image_filename)

    # Load the image
    frame = cv2.imread(image_path)

    # Perform gender detection on the whole frame using YOLOv8
    results = gender_model(frame)[0]

    # Process each detected object (assuming they are all faces)
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            # Extract the region where the YOLO model detected an object
            detected_region = frame[int(y1):int(y2), int(x1):int(x2)]

            if detected_region.size == 0:
                continue

            try:
                # Detect face using MTCNN within the detected region
                faces = face_detector.detect_faces(cv2.cvtColor(detected_region, cv2.COLOR_BGR2RGB))

                for face in faces:
                    fx, fy, fw, fh = face['box']
                    face_x1 = int(x1) + fx
                    face_y1 = int(y1) + fy
                    face_x2 = face_x1 + fw
                    face_y2 = face_y1 + fh

                    face_subregion = frame[face_y1:face_y2, face_x1:face_x2]

                    if face_subregion.size == 0:
                        continue

                    # Perform age and emotion detection
                    predictions = DeepFace.analyze(face_subregion, actions=['age', 'emotion'], enforce_detection=False)
                    if isinstance(predictions, list):
                        predictions = predictions[0]

                    age = predictions['age']
                    dominant_emotion = predictions['dominant_emotion']

                    # Draw a single bounding box with gender, age, and emotion only on the face
                    cv2.rectangle(frame, (face_x1, face_y1), (face_x2, face_y2), (0, 255, 0), 2)
                    cv2.putText(frame, f'Gender: {results.names[int(class_id)].upper()} {score:.2f}, Age: {age}, Emotion: {dominant_emotion}',
                                (face_x1, face_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            except ValueError as e:
                print(f"Error processing face region at [{int(x1), int(y1), int(x2-x1), int(y2-y1)}]: {e}")

    # Save the processed image
    output_image_path = os.path.join(output_directory, image_filename)
    cv2.imwrite(output_image_path, frame)

    print(f"Processed image saved to {output_image_path}")

print("All images processed successfully.")


OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'D:\\IntegrationXpert\\male & female classification (2)\testing'